# Study Experiment Design Generation

This notebook analyzes the characteristics of different space-filling experiment design generation techniques provide by raxpy for different input spaces.

In [1]:
from typing import Dict, NamedTuple, Callable, List, Optional, Tuple

import os
import pickle

import matplotlib.pyplot as plt
import numpy as np

import raxpy
import raxpy.spaces as s
import raxpy.spaces.complexity as c

import raxpy.does.lhs as lhs_doe
import raxpy.does.random as random_doe
from raxpy.does.maxpro import optimize_design_with_sa

import raxpy.does.measure as measure
from raxpy.does import plots
import raxpy.does.doe 
from raxpy.does.doe import EncodingEnum

#import importlib

#importlib.reload(raxpy.does.doe)
plt.style.use('ggplot')


class DoeTuple(NamedTuple):
    design:lhs_doe.DesignOfExperiment
    measurement_set:measure.DesignMeasurementSet


class DesignStrategy(NamedTuple):
    name:str
    strategy:Callable
    base_name:Optional[str] = None


class DesignStrategyResults(NamedTuple):
    name:str
    results:List[DoeTuple]


def compute_sub_space_allocations(doe, sub_spaces):
    actual_counts = {sub_space:0 for sub_space in sub_spaces}

    # determine the sub-space each data-point belongs to
    def map_point(point):
        active_dim_ids = []

        for dim_id, column_index in doe.input_set_map.items():
            if ~np.isnan(point[column_index]):
                active_dim_ids.append(dim_id)

        active_dim_ids.sort()
        actual_counts[tuple(a for a in active_dim_ids)] += 1

    # compute the subspace each point belongs to
    for point in doe.decoded_input_sets:
        map_point(point)
    
    return actual_counts
        
def meets_portions(doe, expected_counts):
    actual_counts = compute_sub_space_allocations(doe,expected_counts.keys())

    for key in expected_counts.keys():
        if expected_counts[key] != actual_counts[key]:
            print(actual_counts)
            return False

    return True


class AssessmentSet(NamedTuple):
    assessment_name:str
    strategies:List[DesignStrategy]
    problem_size_results:Dict[int, List[DesignStrategyResults]]
    input_space:s.InputSpace
    target_full_sub_space_portions:Optional[Dict[Tuple[str],int]]

    def save(self):
        file_path = f"assessment_{self.assessment_name}_results.pkl"
        with open(file_path,"wb") as f:
            pickle.dump(self, f)

    @staticmethod
    def Initalize(
        assessment_name:str,
        strategies:List[DesignStrategy],
        input_space:s.InputSpace,
        target_full_sub_space_portions:Optional[Dict[Tuple[str],int]] = None,
    ) -> "AssessmentSet":
        file_path = f"assessment_{assessment_name}_results.pkl"
        if os.path.exists(file_path):
            with open(file_path,"rb") as f:
                return pickle.load(f)
        else:
            # create new
            return AssessmentSet(
                assessment_name=assessment_name,
                strategies=strategies,
                input_space=input_space,
                problem_size_results={},
                target_full_sub_space_portions=target_full_sub_space_portions,
            )
        
    def generate_designs(self, number_of_designs: int = 10, set_points: int = (100,), target_full_sub_space_portions=None):
        change = False
        for number_of_points in set_points:
            if number_of_points in self.problem_size_results:
                print(f"At least some results exist for {number_of_points}.")
                results = self.problem_size_results[number_of_points]
                base_results_map = {}
                for r in results:
                    base_results_map[r.name] = r
            else:
                results = []
                self.problem_size_results[number_of_points] = results
                base_results_map = {}
            


            for _, strategy in enumerate(self.strategies):
                if strategy.name in base_results_map:
                    print(f"Skipping designs for strategy: '{strategy.name}', already exists.")
                    continue
                print(f"Generating designs for strategy: '{strategy.name}'")
                change = True
                design_count = 0
                ds_results = DesignStrategyResults(strategy.name,[])
                base_results_map[strategy.name] = ds_results
                results.append(ds_results)

                if strategy.base_name is not None:

                    base_strategy = base_results_map[strategy.base_name]
                    for doe_tuple in base_strategy.results:
                        opt_design = strategy.strategy(doe_tuple.design, doe_tuple.design.encoding)
                        measurement_set = measure.measure_with_all_metrics(opt_design, encoding=EncodingEnum.NONE)
                        ds_results.results.append(DoeTuple(opt_design,measurement_set))

                else:
                    while design_count < number_of_designs:
                        doe = strategy.strategy(self.input_space, number_of_points)

                        if self.target_full_sub_space_portions is None or meets_portions(doe, self.target_full_sub_space_portions):
                            # note that we use decoded values for measurement, 
                            # This allows comparison with all stratgies since some 
                            # generate designs work in decoded space
                            # All input spaces use floats with 0 to 1 bounds so 
                            # dimensional range bias not an issue.
                            measurement_set = measure.measure_with_all_metrics(doe, encoding=EncodingEnum.NONE)
                            ds_results.results.append(DoeTuple(doe,measurement_set))
                            design_count += 1
                            print(f"Created design {design_count} for {strategy.name}")
                        else:
                            print("Skipping design")
        if change:
            self.save()
                
        


## Create Helper Functions

In [2]:

def dict_to_hashable_tuple(d):
    return tuple(sorted(d.items()))

class DesignAssessmentGroups:

    def __init__(self, space, strategy_names):
        self.design_allocations = {}
        self.space = space
        self.strategy_names = strategy_names
        self.sub_spaces = tuple(tuple(l) for l in space.derive_full_subspaces())

    def add_design(self,strategy_name, doe_tuple:DoeTuple):
        
        allocation = dict_to_hashable_tuple(
            compute_sub_space_allocations(doe_tuple.design, self.sub_spaces)
        )
        if allocation not in self.design_allocations:
            self.design_allocations[allocation] = {name:[] for name in self.strategy_names}

        self.design_allocations[allocation][strategy_name].append(doe_tuple)
    
    def print_allocation_counts(self):

        for full_sub_space, design_allocation in self.design_allocations.items():
            
            print(f"{full_sub_space} - {', '.join(str(len(d)) for d in design_allocation.values())}")

    def generate_allocation_point_differences(self, target_sub_space_allocations):
        count_differences = {name:[] for name in self.strategy_names}
        for full_sub_space, design_allocation in self.design_allocations.items():
            # compute point count differences from target

            diff = 0
            for sub_spaces, actual_count in full_sub_space:
                target_count = target_sub_space_allocations[sub_spaces]
                diff += abs(target_count - actual_count)

            diff = diff/2.0 # avoid double counting
            for strat_name, doe_tuples in design_allocation.items():
                for _ in range(len(doe_tuples)):
                    count_differences[strat_name].append(diff)
        
        fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

        axs.violinplot(list(v for v in count_differences.values()),
                        showmeans=False,
                        showmedians=True)
        axs.set_xticks(list(i+1 for i in range(len(self.strategy_names))))
        axs.set_xticklabels(self.strategy_names)

        axs.set_title(f'Allocation Point Count Differences from Target')

        plt.show()

def split_designs_by_subspace_allocations(strategies:List[DesignStrategyResults], space:s.InputSpace) -> DesignAssessmentGroups:

    
    assessment_group = DesignAssessmentGroups(space, strategy_names=list(s.name for s in strategies))

    
    for strategy in strategies:

        for doe_tuple in strategy.results:
            # determine the allocations to sub-spaces
            assessment_group.add_design(strategy.name, doe_tuple)
    

    return assessment_group

In [3]:
def plot_fullsubspace_target_portions(space: s.InputSpace, number_of_points: int = 100):
    subspaces = space.derive_full_subspaces()
    
    values = c.compute_subspace_portions(space, subspaces)
    
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    rects = axs.bar(x=list(i for i in range(len(subspaces))),height=values, tick_label=list(", ".join(subspace) for subspace in subspaces))

    axs.bar_label(rects, labels=list(f"{int(value*100)}% - {round(value*number_of_points)}" for value in values))

    axs.set_ylabel("Portion Percentage")

    axs.set_title(f'Target Portions for {number_of_points} points')
    plt.xticks(rotation=45)
    plt.show()

In [4]:
def get_sub_space_measurements(strategies:List[DesignStrategyResults], dim_list, metric=measure.METRIC_DISCREPANCY):
    results = []
    for strategy in strategies:
        design_results = []
        for doe_tuple in strategy.results:
            assessment = doe_tuple.measurement_set.get_full_sub_design_measurements(dim_list)
            if assessment is not None:
                if metric in assessment.measurements:
                    design_results.append(assessment.measurements[metric])
                else:
                    print(f"Skipping design for strategy '{strategy.name}' since it does not metric '{metric}' ")
        results.append(design_results)

    return results

In [5]:
def plot_sub_space_assessments(strategies:List[DesignStrategyResults], dim_list, metric=measure.METRIC_DISCREPANCY):
    assessment_values = get_sub_space_measurements(strategies, dim_list, metric)

    assessmentic_data = [[t for t in assessment_values[i] ] for i in range(len(strategies))]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    axs.set_title(f'{metric} for {", ".join(dim_list)} full-sub-design (smaller the better)')

    plt.show()
    pass

In [6]:
def generate_basic_plots(strategies:List[DesignStrategyResults]):
    
    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_WEIGHTED_DISCREPANCY] for t in strategy.results ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Weighted Discrepancies (smaller the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_WHOLE_MIN_POINT_DISTANCE] for t in strategy.results ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))

    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Minimum Interpoint Distances (larger the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_WHOLE_MIN_PROJECTED_DISTANCE] for t in strategy.results ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
    print(assessmentic_data)
    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Minimum Projected Distances (larger the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

    assessmentic_data = [[t.measurement_set.measurements[measure.METRIC_AVG_MIN_PROJECTED_DISTANCE] for t in strategy.results ] for strategy in strategies]
    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
    print(assessmentic_data)
    axs.violinplot(assessmentic_data,
                    showmeans=False,
                    showmedians=True)
    axs.set_title('Average Minimum Dimension Distances (larger the better)')
    axs.set_xticks(list(i+1 for i in range(len(strategies))))
    axs.set_xticklabels(list(strategy.name for strategy in strategies))

    plt.show()

In [7]:
def peek_into_results(strategies:List[DesignStrategyResults]):

    for strategy in strategies:
        print("\n\n-----------------------------------")
        print(f"Strategy: {strategy.name}")
        doe_tuple = strategy.results[0]
        print("-Example Design Peak-------------------")
        print(doe_tuple.design.decoded_input_sets[0:5,:])
        print("-Measurements-------------------")
        print(doe_tuple.measurement_set.measurements)
        print("-Sub-Design Assessments----------------")
        print(doe_tuple.measurement_set.full_sub_design_measurements)
        

In [8]:
def plot_designs(strategies:List[DesignStrategyResults],design_index=0):
    
    for strategy in strategies:
        design = strategy.results[design_index].design
        fig = plots.plot_scatterplot_matrix(design.decoded_input_sets, list(design.input_set_map.keys()))
        fig.suptitle(strategy.name)

In [9]:


def add_maxpro_strategy_variations(stratgies:List[DesignStrategyResults]):
    s_len = len(stratgies)
    for i in range(s_len):
        base_strategy = stratgies[i]

        new_dsr = DesignStrategyResults(
            strategy=lambda x:x,
            name=f"{base_strategy.name}+MaxPro",
            designs=[]
        )
        for doe_tuple in base_strategy.results:
            opt_design = optimize_design_with_sa(doe_tuple.design, encoding=EncodingEnum.NONE)
            measurement_set = measure.measure_with_all_metrics(opt_design, encoding=EncodingEnum.NONE)
            new_dsr.results.append(DoeTuple(opt_design,measurement_set))
        stratgies.append(new_dsr)

def wrap_with_maxpro(design_f, maxiter=10000):

    def closure_generate_design(space, number_of_points):
        base_design = design_f(space, number_of_points)
        opt_design = optimize_design_with_sa(base_design, maxiter=maxiter)
        return opt_design


    return closure_generate_design
        

# Specify design algorithms to compare

In [10]:
strategies = [
    DesignStrategy("Random", random_doe.generate_design),
    DesignStrategy("FSS-Random", random_doe.generate_seperate_designs_by_full_subspace),
    DesignStrategy("FSS-LHD",lhs_doe.generate_seperate_designs_by_full_subspace),
    DesignStrategy("FSS-LHD-VP",lhs_doe.generate_seperate_designs_by_full_subspace_and_pool),
    DesignStrategy("TT-LHD", lhs_doe.generate_design),
    DesignStrategy("P-LHD", lhs_doe.generate_design_with_projection),
    DesignStrategy("Random+MP", optimize_design_with_sa, "Random"),
    DesignStrategy("FSS-Random-MP", optimize_design_with_sa, "FSS-Random"),
    DesignStrategy("FSS-LHD-MP", optimize_design_with_sa, "FSS-LHD"),
    DesignStrategy("FSS-LHD-VP-MP", optimize_design_with_sa, "FSS-LHD-VP"),
    DesignStrategy("TT-LHD-MP", optimize_design_with_sa, "TT-LHD"),
    DesignStrategy("P-LHD-MP", optimize_design_with_sa, "P-LHD"),
]

# Assessment A: 3 Optional Floats

In [11]:
basic_assessment = AssessmentSet.Initalize(
    assessment_name="basic",
    strategies = strategies,
    input_space= s.InputSpace(
        dimensions=[
            s.Float(id="x1", lb=0.0, ub=1.0, nullable=True, portion_null=(1/3)),
            s.Float(id="x2", lb=0.0, ub=1.0, nullable=True, portion_null=(1/3)),
            s.Float(id="x3", lb=0.0, ub=1.0, nullable=True, portion_null=(1/3)),
        ]
    ),
    # target_sub_space_allocations = {
    #     tuple():0,
    #     ("x1",):1,
    #     ("x2",):1,
    #     ("x3",):1,
    #     ("x1","x2"):8,
    #     ("x1","x3"):8,
    #     ("x2","x3"):8,
    #     ("x1","x2","x3"):73,
    # },
    # small_target_sub_space_allocations = {
    #     tuple():0,
    #     ("x1",):1,
    #     ("x2",):1,
    #     ("x3",):1,
    #     ("x1","x2"):3,
    #     ("x1","x3"):3,
    #     ("x2","x3"):3,
    #     ("x1","x2","x3"):12,
    # }
)

By default when creating dimensions, the target portion of values in a design to be null is unspecified. Creating a design without specifying these values, results in the whole design to have parameters. The following code assigns these portitions using a complexity analysis hueristic.

In [ ]:
plot_fullsubspace_target_portions(basic_assessment.input_space, number_of_points=36)

In [ ]:
basic_assessment.generate_designs(
    number_of_designs=30, set_points=(12, 24, 36)
)

In [ ]:
plot_designs(basic_assessment.problem_size_results[12], design_index =1)

In [ ]:
assessment_group = split_designs_by_subspace_allocations(basic_assessment.problem_size_results[24], basic_assessment.input_space)
assessment_group.print_allocation_counts()
#assessment_group.generate_allocation_point_differences(target_sub_space_allocations)

In [ ]:
generate_basic_plots(basic_assessment.problem_size_results[12])

In [ ]:
generate_basic_plots(basic_assessment.problem_size_results[24])

In [ ]:
generate_basic_plots(basic_assessment.problem_size_results[36])

In [19]:
# custom assessment plots
# plot_sub_space_assessments(strategies, ("x1","x2","x3"), measure.METRIC_DISCREPANCY)
# plot_sub_space_assessments(strategies, ("x1","x2"), measure.METRIC_DISCREPANCY)
# plot_sub_space_assessments(strategies, ["x2","x3"], measure.METRIC_DISCREPANCY)
# plot_sub_space_assessments(strategies, ["x1","x3"], measure.METRIC_DISCREPANCY)
# plot_sub_space_assessments(strategies, ["x1"], measure.METRIC_DISCREPANCY)
# plot_sub_space_assessments(strategies, ["x2"], measure.METRIC_DISCREPANCY)
# plot_sub_space_assessments(strategies, ["x3"], measure.METRIC_DISCREPANCY)

# Assessment B: Simple Heirarchy

In [ ]:
assessment_simple_heirarchy = AssessmentSet.Initalize(
    assessment_name="simple_heirarchy",
    strategies=strategies,
    input_space=s.InputSpace(
        dimensions=[
            s.Float(id="x1", lb=0.0, ub=1.0, nullable=False, portion_null=0.0),
            s.Float(id="x2", lb=0.0, ub=1.0, nullable=True, portion_null=(1/3)),
            s.Composite(id="x3", nullable=True, children=[
                s.Float(id="x4", lb=0.0, ub=1.0, nullable=False, portion_null=0.0),
                s.Float(id="x5", lb=0.0, ub=1.0, nullable=True, portion_null=(1/3)),
            ], portion_null=(1/3))
        ]
    ),
    # target_full_sub_space_portions_bh={
    #     ("x1","x2","x3"):17,
    #     ("x1","x2","x4","x4_1","x4_2"):14,
    #     ("x1","x2","x4","x4_1"):5,
    #     ("x1","x2","x3","x4","x4_1","x4_2"):45,
    #     ("x1","x2","x3","x4","x4_1"):14,
    #     ("x1","x2"):5
    # }
)

number_of_points = 20
plot_fullsubspace_target_portions(assessment_simple_heirarchy.input_space, number_of_points)



In [ ]:
assessment_simple_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(20,),
)

In [ ]:
assessment_simple_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(40,),
)

In [ ]:
assessment_simple_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(60,),
)

In [ ]:
generate_basic_plots(assessment_simple_heirarchy.problem_size_results[20])

In [ ]:
generate_basic_plots(assessment_simple_heirarchy.problem_size_results[40])

In [ ]:
generate_basic_plots(assessment_simple_heirarchy.problem_size_results[60])

# Assessment C: More complex hierarchy

In [ ]:
assessment_modest_heirarchy = AssessmentSet.Initalize(
    assessment_name="modest_heirarchy",
    strategies=strategies,
    input_space = s.InputSpace(
        dimensions=[
            s.Float(id="x1", lb=0.0, ub=1.0, portion_null=0.0),
            s.Float(
                id="x2",
                lb=0.0,
                ub=1.0,
                nullable=True,
                portion_null=1.0 / 3.0,
            ),
            s.Composite(
                id="x3",
                nullable=True,
                portion_null=1.0 / 3.0,
                children=[
                    s.Float(
                        id="x4",
                        lb=0.0,
                        ub=1.0,
                        portion_null=0.0,
                    ),
                    s.Float(
                        id="x5",
                        lb=0.0,
                        ub=1.0,
                        nullable=True,
                        portion_null=1.0 / 3.0,
                    ),
                ],
            ),
            s.Composite(
                id="x6",
                nullable=True,
                portion_null=1.0 / 3.0,
                children=[
                    s.Float(
                        id="x7",
                        lb=0.0,
                        ub=1.0,
                        portion_null=0.0,
                    ),
                    s.Float(
                        id="x8",
                        lb=0.0,
                        ub=1.0,
                        nullable=True,
                        portion_null=1.0 / 3.0,
                    ),
                ],
            ),
            # s.Variant(
            #     id="x6",
            #     nullable=True,
            #     portion_null=0.33,
            #     options=[
            #         s.Float(id="x7", lb=0.0, ub=1.0),
            #         s.Float(id="x8", lb=0.0, ub=1.0),
            #     ],
            # ),
        ]
    )
)


plot_fullsubspace_target_portions(assessment_modest_heirarchy.input_space, number_of_points=32)

In [ ]:
assessment_modest_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(32,),
)

In [ ]:
assessment_modest_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(64,),
)

In [ ]:
assessment_modest_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(96,),
)

In [ ]:
generate_basic_plots(assessment_modest_heirarchy.problem_size_results[32])

In [ ]:
generate_basic_plots(assessment_modest_heirarchy.problem_size_results[64])

In [ ]:
generate_basic_plots(assessment_modest_heirarchy.problem_size_results[96])

In [ ]:
plot_designs(assessment_simple_heirarchy.problem_size_results[64],design_index =1)

In [ ]:
peek_into_results(assessment_simple_heirarchy.problem_size_results[64])

In [ ]:
assessment_complex_heirarchy = AssessmentSet.Initalize(
    assessment_name="complex_heirarchy",
    strategies=strategies,
    input_space = s.InputSpace(
        dimensions=[
            s.Float(id="x1", lb=0.0, ub=1.0, portion_null=0.0),
            s.Float(
                id="x2",
                lb=0.0,
                ub=1.0,
                nullable=True,
                portion_null=1.0 / 3.0,
            ),
            s.Composite(
                id="x3",
                nullable=True,
                portion_null=1.0 / 3.0,
                children=[
                    s.Float(
                        id="x4",
                        lb=0.0,
                        ub=1.0,
                        portion_null=0.0,
                    ),
                    s.Composite(
                        id="x5",
                        nullable=True,
                        portion_null=1.0 / 3.0,
                        children=[
                            s.Float(
                                id="x6",
                                lb=0.0,
                                ub=1.0,
                                portion_null=0.0,
                            ),
                            s.Float(
                                id="x7",
                                lb=0.0,
                                ub=1.0,
                                nullable=True,
                                portion_null=1.0 / 3.0,
                            ),
                        ],
                    ),
                ],
            ),
            s.Composite(
                id="x8",
                nullable=True,
                portion_null=1.0 / 3.0,
                children=[
                    s.Float(
                        id="x9",
                        lb=0.0,
                        ub=1.0,
                        portion_null=0.0,
                    ),
                    s.Float(
                        id="x10",
                        lb=0.0,
                        ub=1.0,
                        nullable=True,
                        portion_null=1.0 / 3.0,
                    ),
                ],
            ),
            # s.Variant(
            #      id="x11",
            #      nullable=True,
            #      portion_null=0.33,
            #      options=[
            #          s.Float(id="x12", lb=0.0, ub=1.0, portion_null=0.0),
            #          s.Float(id="x13", lb=0.0, ub=1.0, portion_null=0.0),
            #      ],
            # ),
        ]
    )
)

plot_fullsubspace_target_portions(assessment_complex_heirarchy.input_space, number_of_points=10*4)

In [ ]:
assessment_complex_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(10*4,),
)

In [ ]:
assessment_complex_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(10*4*2,),
)

In [ ]:
assessment_complex_heirarchy.generate_designs(
    number_of_designs=30,
    set_points=(10*4*3,),
)

In [42]:
def generate_group_assessment_plot_diff(metric, assessments:List[AssessmentSet], strategies1:List[DesignStrategy], strategies2:List[DesignStrategy]):

    # Group labels for the x-axis
    group_labels = []
    groups = []

    for assessment in assessments:
        for problem_size, results in assessment.problem_size_results.items():
            group_labels.append(f"{assessment.assessment_name}({problem_size})")
            max_v = -np.inf
            min_v = np.inf
            group_data = []
            for strategy1, strategy2 in zip(strategies1,strategies2):
                values = []
                for r1 in results:
                    if r1.name == strategy1.name:
                        break
                for r2 in results:
                    if r2.name == strategy2.name:
                        break
                for doe_tuple1, doe_tuple2 in zip(r1.results, r2.results):
                    value1 = doe_tuple1.measurement_set.measurements[metric]
                    value2 = doe_tuple2.measurement_set.measurements[metric]
                    value = value2 - value1
                    if value > max_v:
                        max_v = value
                    if value < min_v:
                        min_v = value
                    values.append(value)
                group_data.append(np.array(values))
            # normalize

            if min_v < 0:
                if max_v > abs(min_v):
                    min_v = max_v * -1
            elif min_v > 0:
                min_v = 0
            if max_v < 0:
                max_v = 0.0

            for data in group_data:
                data[:] = data  /abs(max_v - min_v)
            groups.append(group_data)

    # Data as [problem1 Strategy 1, problem 2  stragey 2], [Group B1, Group B2]
    all_data = []
    positions = []
    x_ticks = []
    x_tick_labels = []
    p_index = 0
    color_map = [(1,0,0), (0.8,0,0), (0.6,0,0),
                 (0,1,0), (0,0.8,0), (0,0.6,0),
                 (0,0,1), (0,0,0.8), (0,0,0.6),
                 (1,1,0), (0.8,0.8,0), (0.6,0.6,0)]
    colors = []
    for i, strategy in enumerate(strategies1):
        p_index += 2
        x_ticks.append(p_index + len(groups)/2 - 0.5)
        x_tick_labels.append(strategy.name)
        for j, group_data in enumerate(groups):
            colors.append(color_map[j])
            positions.append(p_index)
            data = group_data[i]
            all_data.append(data)
            p_index += 1


    # Create a figure and axis
    fig, ax = plt.subplots()
    fig.set_figwidth(12)

    # Grouped boxplot
    box = ax.boxplot(all_data, patch_artist=True, positions=positions)

    # Add colors to differentiate between groupings
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)

    # Adding a legend for the groupings
    handles = list(plt.Line2D([0], [0], color=color, lw=4, label=lbl) for color, lbl in zip(colors, group_labels))

    ax.legend(handles=handles,loc='center left',title="Input Space (problem size)", bbox_to_anchor=(1,0.5))


    # Adjust the x-axis labels
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_tick_labels)

    # Adding title and labels
    #ax.set_title('Grouped Boxplot with Additional Grouping Factor')
    ax.set_xlabel('Design Algorithm')
    ax.set_ylabel('Normalized Difference')

    for i in range(1,len(strategies1)):
        plt.axvline(x=len(groups)*i+2*i+0.5, color='black', linestyle='--', lw=1)


    # Adjust layout to fit legend
    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Make space for the legend on the right


    # Display the plot
    plt.show()

In [43]:
def generate_group_assessment_plot(metric, assessments:List[AssessmentSet], strategies:List[DesignStrategy]):

    # Group labels for the x-axis
    group_labels = []
    groups = []

    for assessment in assessments:
        for problem_size, results in assessment.problem_size_results.items():
            group_labels.append(f"{assessment.assessment_name}({problem_size})")
            max_v = -np.inf
            min_v = np.inf
            group_data = []
            for strategy in strategies:
                values = []
                for r in results:
                    if r.name == strategy.name:
                        break
                for doe_tuple in r.results:
                    value = doe_tuple.measurement_set.measurements[metric]
                    if value > max_v:
                        max_v = value
                    if value < min_v:
                        min_v = value
                    values.append(value)
                group_data.append(np.array(values))
            # normalize
            for data in group_data:
                data[:] = (data - min_v)  /(max_v - min_v)
            groups.append(group_data)

    # Data as [problem1 Strategy 1, problem 2  stragey 2], [Group B1, Group B2]
    all_data = []
    positions = []
    x_ticks = []
    x_tick_labels = []
    p_index = 0
    color_map = [(1,0,0), (0.8,0,0), (0.6,0,0),
                 (0,1,0), (0,0.8,0), (0,0.6,0),
                 (0,0,1), (0,0,0.8), (0,0,0.6),
                 (1,1,0), (0.8,0.8,0), (0.6,0.6,0)]
    colors = []
    for i, strategy in enumerate(strategies):
        p_index += 2
        x_ticks.append(p_index + len(groups)/2 - 0.5)
        x_tick_labels.append(strategy.name)
        for j, group_data in enumerate(groups):
            colors.append(color_map[j])
            positions.append(p_index)
            data = group_data[i]
            all_data.append(data)
            p_index += 1


    # Create a figure and axis
    fig, ax = plt.subplots()
    fig.set_figwidth(12)

    # Grouped boxplot
    box = ax.boxplot(all_data, patch_artist=True, positions=positions)

    # Add colors to differentiate between groupings
    for patch, color in zip(box['boxes'], colors):
        patch.set_facecolor(color)

    # Adding a legend for the groupings
    handles = list(plt.Line2D([0], [0], color=color, lw=4, label=lbl) for color, lbl in zip(colors, group_labels))

    ax.legend(handles=handles,loc='center left',title="Input Space (problem size)", bbox_to_anchor=(1,0.5))


    # Adjust the x-axis labels
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_tick_labels, rotation=45)

    # Adding title and labels
    #ax.set_title('Grouped Boxplot with Additional Grouping Factor')
    ax.set_xlabel('Design Algorithm')
    ax.set_ylabel('Normalized Value')

    for i in range(1,len(strategies)):
        plt.axvline(x=len(groups)*i+2*i+0.5, color='black', linestyle='--', lw=1)


    # Adjust layout to fit legend
    plt.tight_layout(rect=[0, 0, 0.9, 1])  # Make space for the legend on the right


    # Display the plot
    plt.show()

In [44]:
assessments = [
    basic_assessment,
    assessment_simple_heirarchy,
    assessment_modest_heirarchy,
    assessment_complex_heirarchy
]

strategies_w_mp = list(s for s in strategies if s.name.find("MP") >= 0)
strategies_wo_mp = list(s for s in strategies if s.name.find("MP") < 0)

In [ ]:
generate_group_assessment_plot_diff(measure.METRIC_WHOLE_MIN_POINT_DISTANCE, assessments,strategies_wo_mp, strategies_w_mp)

In [ ]:
generate_group_assessment_plot_diff(measure.METRIC_WEIGHTED_DISCREPANCY, assessments,strategies_wo_mp, strategies_w_mp)

In [ ]:

generate_group_assessment_plot_diff(measure.METRIC_WHOLE_STAR_DISCREPANCY, assessments,strategies_wo_mp, strategies_w_mp)

In [ ]:
generate_group_assessment_plot(measure.METRIC_WHOLE_MIN_POINT_DISTANCE, assessments, strategies)

In [ ]:
generate_group_assessment_plot(measure.METRIC_WHOLE_STAR_DISCREPANCY, assessments, strategies)

In [ ]:
generate_group_assessment_plot(measure.METRIC_WEIGHTED_DISCREPANCY, assessments, list(strategy for strategy in strategies if strategy.name.find("FSS") >= 0))

In [ ]:
generate_group_assessment_plot(measure.METRIC_AVG_MIN_PROJECTED_DISTANCE, assessments, list(strategy for strategy in strategies if strategy.name.find("MP") < 0))

In [ ]:
peek_into_results(strategies_ch_2)

In [ ]:
generate_basic_plots(strategies_ch_2)

In [ ]:
plot_fullsubspace_target_portions(space_ch,number_of_points=30)

In [ ]:
plot_designs(strategies_ch_2,design_index =1)